In [ ]:
train_test_split = np.random.rand(len(features)) < 0.70 # randomly divide training sets and test sets
train_x = features[train_test_split]
train_y = labels[train_test_split]
test_x = features[~train_test_split]
test_y = labels[~train_test_split]

In [ ]:
class_num = label_num
n_hidden_units_one = 200
n_hidden_units_two = 50
#30/20 for 9 points fft
#120/60 for 16 fft
#200/100 for 16 light + heavy fft

# n_hidden_units_three = 80
learning_rate = 0.01
n_dim = features.shape[1]
sd = 1/np.sqrt(n_dim)

# Input Layer
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, n_dim],name='x_input')
    y_true = tf.placeholder(tf.float32, [None,class_num],name='y_input')

# Hidden Layers
with tf.name_scope('hidden_layer_1'):
    with tf.name_scope('W1'):
        W_1 = tf.Variable(tf.random_normal([n_dim,n_hidden_units_one], mean=0, stddev=sd, name='Weights'))
    with tf.name_scope('b1'):
        b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean=0, stddev=sd, name='biases'))
    with tf.name_scope('h1'):
        h_1 = tf.nn.tanh(tf.matmul(x, W_1) + b_1, name='output')

with tf.name_scope('hidden_layer_2'):
    with tf.name_scope('W2'):
        W_2 = tf.Variable(tf.random_normal([n_hidden_units_one,n_hidden_units_two], mean=0, stddev=sd, name='Weights'))
    with tf.name_scope('b2'):
        b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean=0, stddev=sd, name='biases'))
    with tf.name_scope('h2'):
        h_2 = tf.nn.sigmoid(tf.matmul(h_1, W_2) + b_2, name='output')
        
# with tf.name_scope('hideen_layer_3'):
#     with tf.name_scope('W3'):
#         W_3 = tf.Variable(tf.random_normal([n_hidden_units_two,n_hidden_units_three], mean=0, stddev=sd, name='Weights'))
#     with tf.name_scope('b3'):
#         b_3 = tf.Variable(tf.random_normal([n_hidden_units_three], mean=0, stddev=sd, name='biases'))
#     with tf.name_scope('h3'):
#         h_3 = tf.nn.sigmoid(tf.matmul(h_2, W_3) + b_3, name='output')

# Output Layer
with tf.name_scope('output_layer'):
    with tf.name_scope('W'):
        W = tf.Variable(tf.random_normal([n_hidden_units_two,class_num], mean=0, stddev=sd, name='Weights'))
    with tf.name_scope('b'):
        b = tf.Variable(tf.random_normal([class_num], mean=0, stddev=sd, name='biases'))
    with tf.name_scope('output'):
        y = tf.nn.softmax(tf.matmul(h_2, W) + b, name='output')

# Cost Function
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true*tf.log(y), reduction_indices=[1]))
#     cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y))
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    print(train_step)

In [ ]:
from tensorflow.python.framework import graph_util

In [ ]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)
    
for step in range(200):
    batch_x, batch_y = train_x,train_y
    train_step.run({x:batch_x, y_true:batch_y})
    if step%10==0:
#         y_res = tf.identity(y,name='y_prediction') # Output
        pre_num=tf.argmax(y,1,output_type='int32',name='output')
        correct_prediction = tf.equal(pre_num,tf.argmax(y_true,1,output_type='int32'))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        a = accuracy.eval({x:test_x,y_true:test_y})
        print('Accuracy：{0}'.format(a))
        
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def,output_node_names=['output_layer/output/output'])
with tf.gfile.FastGFile('../../data/VibAuthModel.pb', mode='wb') as f:
    f.write(output_graph_def.SerializeToString())
sess.close()